# More on Keras Models

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from sklearn import metrics as sk_metrics
from sklearn import linear_model as sk_lm
from sklearn import model_selection as sk_msel
from sklearn import discriminant_analysis as sk_lda

Using TensorFlow backend.


### Load test data

In [9]:
%%bash
curl -O 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 58988  100 58988    0     0  41984      0  0:00:01  0:00:01 --:--:-- 41954


In [2]:
# Test Data
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
concrete_data.shape

(1030, 9)

In [4]:
# Split in predictors and target dataframes
predictors=concrete_data.drop(columns=['Strength'])
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [5]:
target=concrete_data['Strength']
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [6]:
# Number of predictors
n_predictors=len(predictors.columns)
n_predictors

8

In [7]:
# normalize predictors
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [12]:
# Build the neural network
model=keras.models.Sequential()
model.add(keras.layers.Dense(50, activation='relu', input_shape=(n_predictors,)))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [13]:
# Fit the model
model.fit(x=predictors_norm, y=target, validation_split=0.3, epochs=100, verbose=2)

Train on 721 samples, validate on 309 samples
Epoch 1/100
 - 0s - loss: 1608.7039 - val_loss: 1113.3599
Epoch 2/100
 - 0s - loss: 1436.0887 - val_loss: 974.9112
Epoch 3/100
 - 0s - loss: 1182.2193 - val_loss: 777.5706
Epoch 4/100
 - 0s - loss: 843.6380 - val_loss: 550.4068
Epoch 5/100
 - 0s - loss: 517.2519 - val_loss: 358.4077
Epoch 6/100
 - 0s - loss: 310.9301 - val_loss: 249.1133
Epoch 7/100
 - 0s - loss: 246.2803 - val_loss: 206.1512
Epoch 8/100
 - 0s - loss: 221.6555 - val_loss: 195.2260
Epoch 9/100
 - 0s - loss: 208.4810 - val_loss: 187.5345
Epoch 10/100
 - 0s - loss: 199.5777 - val_loss: 181.9799
Epoch 11/100
 - 0s - loss: 192.4997 - val_loss: 180.8424
Epoch 12/100
 - 0s - loss: 186.3305 - val_loss: 178.4709
Epoch 13/100
 - 0s - loss: 181.7568 - val_loss: 172.5118
Epoch 14/100
 - 0s - loss: 177.4673 - val_loss: 172.6840
Epoch 15/100
 - 0s - loss: 173.5824 - val_loss: 169.7838
Epoch 16/100
 - 0s - loss: 169.6344 - val_loss: 168.6778
Epoch 17/100
 - 0s - loss: 167.0175 - val_loss:

### Stop early when validation metric settled

In [9]:
# Define callback list
cb = [keras.callbacks.EarlyStopping(patience=5)]

In [10]:
# Fit the model with stop early callback
model.fit(x=predictors_norm, y=target, validation_split=0.3, epochs=100, verbose=2, callbacks=cb)

Train on 721 samples, validate on 309 samples
Epoch 1/100
 - 0s - loss: 1685.5080 - val_loss: 1195.9776
Epoch 2/100
 - 0s - loss: 1642.5102 - val_loss: 1152.2916
Epoch 3/100
 - 0s - loss: 1584.8898 - val_loss: 1092.2270
Epoch 4/100
 - 0s - loss: 1499.4835 - val_loss: 1009.3925
Epoch 5/100
 - 0s - loss: 1375.7122 - val_loss: 898.7299
Epoch 6/100
 - 0s - loss: 1204.9545 - val_loss: 760.7385
Epoch 7/100
 - 0s - loss: 990.7571 - val_loss: 613.5238
Epoch 8/100
 - 0s - loss: 762.8075 - val_loss: 474.0332
Epoch 9/100
 - 0s - loss: 551.3970 - val_loss: 368.6660
Epoch 10/100
 - 0s - loss: 393.8899 - val_loss: 303.7995
Epoch 11/100
 - 0s - loss: 303.1165 - val_loss: 269.7237
Epoch 12/100
 - 0s - loss: 259.6393 - val_loss: 252.2754
Epoch 13/100
 - 0s - loss: 235.9681 - val_loss: 241.7144
Epoch 14/100
 - 0s - loss: 224.3578 - val_loss: 234.0749
Epoch 15/100
 - 0s - loss: 215.7804 - val_loss: 229.3233
Epoch 16/100
 - 0s - loss: 209.5733 - val_loss: 222.6845
Epoch 17/100
 - 0s - loss: 204.1758 - val

### Model Definition with Functional API

In [17]:
# Define same model as above with functional API

# Define layers
i = keras.layers.Input(shape=(n_predictors,), name='Input-Layer')
l1 = keras.layers.Dense(50, activation='relu', name='Hidden-1')(i)
l2 = keras.layers.Dense(50, activation='relu', name='Hidden-2')(l1)
o = keras.layers.Dense(1, name='Output-Layer')(l2)

model=keras.models.Sequential()
model.add(keras.layers.Dense(50, activation='relu', input_shape=(n_predictors,)))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(1))

# Define model
model2 = keras.models.Model(inputs=i, outputs=o, name='Model_2')

In [19]:
# Summary of model defined by Sequential()
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 50)                450       
_________________________________________________________________
dense_14 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 51        
Total params: 3,051
Trainable params: 3,051
Non-trainable params: 0
_________________________________________________________________


In [20]:
# And model2 defined by functional API
model2.summary()

Model: "Model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input-Layer (InputLayer)     (None, 8)                 0         
_________________________________________________________________
Hidden-1 (Dense)             (None, 50)                450       
_________________________________________________________________
Hidden-2 (Dense)             (None, 50)                2550      
_________________________________________________________________
Output-Layer (Dense)         (None, 1)                 51        
Total params: 3,051
Trainable params: 3,051
Non-trainable params: 0
_________________________________________________________________
